In [1]:
import pandas as pd
import os
import ast
import numpy as np
import tensorflow as tf
import einops

from keras.layers import Input, Dense, concatenate, Embedding, ReLU, Dropout, Bidirectional, GRU, Add, Concatenate
from keras import Model

In [2]:
UNITS = 256

In [3]:
os.getcwd()

'c:\\Users\\salha\\studium\\NLP\\projektabeit_nlp_2\\analyse_phase_01'

In [4]:
text_path = os.path.join('data', 'choc_recipes.txt')
# Using readlines()
file1 = open(text_path, 'r')
Lines = file1.readlines()

names = []
ingredients = []
steps = []

count = 0
# Strips the newline character
for line in Lines:
    pieces = line.split('\t')
    names.append(pieces[0])
    ingredients.append(pieces[1])
    steps.append(pieces[2])




In [5]:
print(f'Name count: {len(names)}')
print(f'Ingredient count: {len(ingredients)}')
print(f'Step count: {len(steps)}')

Name count: 7404
Ingredient count: 7404
Step count: 7404


In [6]:
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    parsed = einops.parse_shape(tensor, names)

    for name, new_dim in parsed.items():
      old_dim = self.shapes.get(name, None)

      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

In [7]:
ingredients_list_list = []
for i_string in ingredients:
    i = i_string.replace(', ', '*')
    i = i.replace(' ', '_')
    i = i.replace('*', ', ')

    ingredients_list_list.append(i)

In [8]:
ingredients = ingredients_list_list

In [9]:
BUFFER_SIZE = 120
BATCH_SIZE = 64

name_input = np.array(names)
ingredient_input = np.array(ingredients)
step_output = np.array(steps)

ingredient_dataset = (tf.data.Dataset.from_tensor_slices(ingredient_input).shuffle(BUFFER_SIZE).batch(BATCH_SIZE))
name_dataset =  (tf.data.Dataset.from_tensor_slices(name_input).shuffle(BUFFER_SIZE).batch(BATCH_SIZE))
step_dataset =  (tf.data.Dataset.from_tensor_slices(step_output).shuffle(BUFFER_SIZE).batch(BATCH_SIZE))
#dataset_12 = tf.data.Dataset.from_tensor_slices((name_input, ingredient_input))
#dataset_label = tf.data.Dataset.from_tensor_slices(step_output)

#dataset = tf.data.Dataset.zip((dataset_12, dataset_label)).batch(64)
#model.fit(dataset, epochs=10, steps_per_epoch=4)



In [43]:
is_train = np.random.uniform(size=(len(step_dataset),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((name_dataset[is_train], step_dataset[is_train]))
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((name_dataset[~is_train], step_dataset[~is_train]))
    .batch(BATCH_SIZE))

TypeError: 'BatchDataset' object is not subscriptable

In [10]:
max_features = 10000
max_len = 400


step_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)
step_vectorize_layer.adapt(step_dataset)

In [11]:
max_features = 2000
max_len = 4


ingredient_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)

ingredient_vectorize_layer.adapt(ingredients)

In [12]:
max_features = 5000
max_len = 4


name_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)
name_vectorize_layer.adapt(name_dataset)

In [13]:
name_vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'chocolate',
 'cake',
 'cookies',
 'chip',
 's',
 'brownies',
 'fudge',
 'butter',
 'peanut',
 'bars',
 'cream',
 'pie',
 'with',
 'and',
 'white',
 'hot',
 'pudding',
 'brownie',
 'muffins',
 'easy',
 'no',
 'cheesecake',
 'oatmeal',
 'banana',
 'double',
 'mocha',
 'cocoa',
 'cookie',
 'cupcakes',
 'caramel',
 'bread',
 'the',
 'free',
 'cherry',
 'low',
 'bake',
 'orange',
 'mousse',
 'frosting',
 'coffee',
 'ice',
 'fat',
 'squares',
 'sauce',
 'coconut',
 'almond',
 'best',
 'dark',
 'mix',
 'dessert',
 'raspberry',
 'mint',
 'toffee',
 'chewy',
 'cheese',
 'pecan',
 'vegan',
 'for',
 'black',
 'a',
 'rich',
 'nut',
 'truffles',
 'balls',
 'fudgy',
 'candy',
 'layer',
 'mexican',
 'triple',
 'or',
 'biscotti',
 'walnut',
 'of',
 'chunk',
 'bar',
 'cranberry',
 'cinnamon',
 'in',
 'decadent',
 'bundt',
 'kahlua',
 'milk',
 'creamy',
 'cakes',
 'marshmallow',
 'espresso',
 'sugar',
 'oreo',
 'torte',
 'pumpkin',
 'gluten',
 'strawberry',
 'one',
 'icing',
 'mini',
 '

In [14]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, name_processor, ingredient_processor, units):
    super(Encoder, self).__init__()
    
    self.name_processor = name_processor
    self.ingredient_processor = ingredient_processor

    self.name_vocab_size = name_processor.vocabulary_size()
    self.ingredient_vocab_size = ingredient_processor.vocabulary_size()
    
    self.units = units

    # The embedding layer converts tokens to vectors
    self.name_embedding = tf.keras.layers.Embedding(self.name_vocab_size, units, mask_zero=True)
    self.ingredient_embedding = tf.keras.layers.Embedding(self.ingredient_vocab_size, units, mask_zero=True)

    self.concat = Concatenate(axis=-1)

    self.embedding = tf.keras.layers.Embedding((self.name_vocab_size + self.ingredient_vocab_size), units, mask_zero=True)
    #self.embedding = tf.keras.layers.Embedding(self.name_vocab_size, units, mask_zero=True)

    # The RNN layer processes those vectors sequentially.
    self.rnn = tf.keras.layers.Bidirectional(
        merge_mode='sum',
        layer=tf.keras.layers.GRU(units,
                            # Return the sequence and state
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform'))

  def call(self, ingredients, names):

    shape_checker = ShapeChecker()
    #shape_checker(x, 'batch s')

    # 2. The embedding layer looks up the embedding vector for each token.
    x1 = self.name_embedding(names)
    print(f'Dimensions of x1: {x1.shape}')
    x2 = self.ingredient_embedding(ingredients)
    print(f'Dimensions of x2: {x2.shape}')
    x = self.concat([x1, x2])
    #print(f'Dimensions of x after concat: {x.shape}')
    #x = x1
    #x = self.embedding(x)

    #shape_checker(x, 'batch s units')
    
    print(f'Dimensions of x after embedding: {x.shape}')


   
    # 3. The GRU processes the sequence of embeddings.
    x = self.rnn(x)
    
    print(f'Dimensions of x after rnn: {x.shape}')
    shape_checker(x, 'batch s units')
  
    # 4. Returns the new sequence of embeddings.
    return x

  def convert_input(self, ingredients, names):
    ingredients = tf.convert_to_tensor(ingredients)
    names = tf.convert_to_tensor(names)
    
    names = self.name_processor(names)#.to_tensor()
    ingredients = self.ingredient_processor(ingredients)#.to_tensor()
    
    context = self(ingredients, names)
    return context

In [ ]:
def process_text(context, target):

  concat = Concatenate(axis=-1)
  name_context = name_vectorize_layer.to_tensor()
  ingredient_context = ingredient_vectorize_layer.to_tensor()
  context = concat([name_context, ingredient_context])
  target = step_vectorize_layer.to_tensor()
  targ_in = target[:,:-1].to_tensor()
  targ_out = target[:,1:].to_tensor()
  return (context, targ_in), targ_out


train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)

In [15]:


# Encode the input sequence.
encoder = Encoder(name_vectorize_layer, ingredient_vectorize_layer, UNITS)



In [16]:
for ex_names in name_dataset.take(1):
  print(ex_names.numpy()) 
ex_names_tok = name_vectorize_layer(ex_names)
print(ex_names_tok)

[b'almost  no fat chocolate cake'
 b'1 gram fat chocolate banana freezer pops'
 b'get the sensation  brownies' b'aim s best batch of brownies'
 b'addicting chocolate chip cookies' b'5 minute chocolate lava cake'
 b'turtle  squares' b'5 layer delicious dessert' b'a different tiramisu'
 b'all recipes white chocolate blondies' b'almond joy cheesecake'
 b'22 minute cake' b'30 minute chocolate cake' b'88 calorie brownies'
 b'almond joy candies' b'almond joy chocolate pie'
 b'addictive white chocolate cookies'
 b'almond chocolate cake with ganache' b'5 chip cookies'
 b'alice pike s famous brownies' b'alice s nanaimo bars' b'1 minute cake'
 b'almond toffee mocha squares' b'aces  frozen chocolate mousse pie'
 b'almond chocolate cookies' b'all american chewy chocolate chip cookies'
 b'almost  starbucks frappuccino' b'agasajos  mexican hot chocolate'
 b'almost instant rocky road'
 b'almond chocolate chip cookies  no baking powder soda'
 b'almond hot chocolate' b'3 layer chocolate peanut butter b

In [17]:
ex_names_tok.shape

TensorShape([64, 4])

In [18]:
for (ex_ingredients) in ingredient_dataset.take(1):
  print(ex_ingredients.numpy()) 

ex_ingredients_tok = ingredient_vectorize_layer(ex_ingredients)
print(ex_ingredients_tok)

[b'coffee, cocoa, cognac, kahlua, vanilla_extract, half-and-half_cream'
 b'milk_chocolate_chips, butterscotch_chips, creamy_peanut_butter, butter, sugar, evaporated_milk, marshmallow_cream, vanilla, honey_roasted_peanuts, caramels, whipping_cream'
 b'butter, brown_sugar, water, coconut, pecans, flour, sugar, cocoa, baking_powder, milk, eggs, vanilla, mini_chocolate_chip'
 b"blanched_slivered_almond, unsalted_butter, confectioners'_sugar, salt, almond_extract, flour, white_chocolate, flaked_coconut"
 b'cake_flour, sugar, baking_cocoa, egg, milk, oil, chocolate_chips, vanilla'
 b"almond_paste, confectioners'_sugar, egg, all-purpose_flour, almonds, bittersweet_chocolate"
 b'sugar, butter, almond_extract, egg, all-purpose_flour, baking_powder, salt, chocolate_chips, powdered_sugar, milk, sliced_almonds'
 b'all-purpose_flour, salt, baking_soda, butter, peanut_butter, white_sugar, brown_sugar, eggs, chocolate_chips'
 b'semi-sweet_chocolate_chips, pine_nuts, red_chili_powder, sweetened_conden

In [19]:
ex_ingredients_tok.shape

TensorShape([64, 4])

In [20]:
for (ex_steps) in step_dataset.take(1):
  print(ex_steps.numpy()) 
ex_steps_tok = step_vectorize_layer(ex_steps)
print(ex_steps_tok)

[b'combine greek yogurt , cocoa powder , and splenda in bowl and stir until well mixed <STEP> stir in mixed berries until well distributed <STEP> top with sprinkles of fiber one cereal\n'
 b'preheat the oven to 400 degrees <STEP> line a cookie sheet with aluminum foil <STEP> i spray it with non-stick cooking spray like pam to help with the sticking , and of course you can always use more butter <STEP> place the crackers side by side on the tray forming a large rectangle <STEP> this will make the caramel and chocolate easier to spread <STEP> over high heat , combine the butter and brown sugar in a saucepan <STEP> stirring constantly , bring to a boil and cook for 3 minutes after boiling <STEP> pour caramel mixture over crackers and spread evenly to the edges <STEP> i use an offset spatula for this <STEP> bake for 5 minutes , pull out and sprinkle the chocolate chips evenly over the crackers <STEP> bake for 3 minutes more , pull out and spread the softened chocolate chips evenly over the

In [21]:
sample_name = name_dataset.take(1)
sample_ingredient = ingredient_dataset.take(1)

# Encode the input sequence.
encoder = Encoder(name_vectorize_layer, ingredient_vectorize_layer, UNITS)
ex_context = encoder(ex_ingredients_tok, ex_names_tok)

#print(f'Context tokens, shape (batch, s): {ex_context_tok.shape}')
print(f'Encoder output, shape (batch, s, units): {ex_context.shape}')

Dimensions of x1: (64, 4, 256)
Dimensions of x2: (64, 4, 256)
Dimensions of x after embedding: (64, 4, 512)
Dimensions of x after rnn: (64, 4, 256)
Encoder output, shape (batch, s, units): (64, 4, 256)


In [22]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):
    shape_checker = ShapeChecker()
 
    #shape_checker(x, 'batch t units')
    #shape_checker(context, 'batch s units')

    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)
    
    #shape_checker(x, 'batch t units')
    #shape_checker(attn_scores, 'batch heads t s')
    
    # Cache the attention scores for plotting later.
    attn_scores = tf.reduce_mean(attn_scores, axis=1)
    #shape_checker(attn_scores, 'batch t s')
    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [23]:
attention_layer = CrossAttention(UNITS)

# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(step_vectorize_layer.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True, name="bliblablub")

ex_tar_embed = embed(ex_steps_tok)
result = attention_layer(ex_tar_embed, ex_context)

print(f'Context sequence, shape (batch, s, units): {ex_context.shape}')
print(f'Target sequence, shape (batch, t, units): {ex_tar_embed.shape}')
print(f'Attention result, shape (batch, t, units): {result.shape}')
print(f'Attention weights, shape (batch, t, s):    {attention_layer.last_attention_weights.shape}')

Context sequence, shape (batch, s, units): (64, 4, 256)
Target sequence, shape (batch, t, units): (64, 400, 256)
Attention result, shape (batch, t, units): (64, 400, 256)
Attention weights, shape (batch, t, s):    (64, 400, 4)


In [24]:
class Decoder(tf.keras.layers.Layer):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, text_processor, units):
    super(Decoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]')
    self.id_to_word = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]',
        invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')

    self.units = units


    # 1. The embedding layer converts token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                               units, mask_zero=True)

    # 2. The RNN keeps track of what's been generated so far.
    self.rnn = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    # 3. The RNN output will be the query for the attention layer.
    self.attention = CrossAttention(units)

    # 4. This fully connected layer produces the logits for each
    # output token.
    self.output_layer = tf.keras.layers.Dense(self.vocab_size)

In [25]:
@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  
  # 1. Lookup the embeddings
  x = self.embedding(x)

  # 2. Process the target sequence.
  x, state = self.rnn(x, initial_state=state)
  # 3. Use the RNN output as the query for the attention over the context.
  x = self.attention(x, context)
  self.last_attention_weights = self.attention.last_attention_weights

  # Step 4. Generate logit predictions for the next token.
  logits = self.output_layer(x)

  if return_state:
    return logits, state
  else:
    return logits

In [26]:
decoder = Decoder(step_vectorize_layer, UNITS)

In [27]:
logits = decoder(ex_context, ex_steps_tok)

print(f'encoder output shape: (batch, s, units) {ex_context.shape}')
print(f'input target tokens shape: (batch, t) {ex_steps_tok.shape}')
print(f'logits shape shape: (batch, target_vocabulary_size) {logits.shape}')

encoder output shape: (batch, s, units) (64, 4, 256)
input target tokens shape: (batch, t) (64, 400)
logits shape shape: (batch, target_vocabulary_size) (64, 400, 8257)


In [28]:
@Decoder.add_method
def get_initial_state(self, context):
  batch_size = tf.shape(context)[0]
  start_tokens = tf.fill([batch_size, 1], self.start_token)
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  embedded = self.embedding(start_tokens)
  return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

In [29]:
@Decoder.add_method
def tokens_to_text(self, tokens):
  words = self.id_to_word(tokens)
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
  result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
  return result

In [30]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):
  logits, state = self(
    context, next_token,
    state = state,
    return_state=True) 

  if temperature == 0.0:
    next_token = tf.argmax(logits, axis=-1)
  else:
    logits = logits[:, -1, :]/temperature
    next_token = tf.random.categorical(logits, num_samples=1)

  # If a sequence produces an `end_token`, set it `done`
  done = done | (next_token == self.end_token)
  # Once a sequence is done it only produces 0-padding.
  next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

  return next_token, done, state

In [31]:
# Setup the loop variables.
next_token, done, state = decoder.get_initial_state(ex_context)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      ex_context, next_token, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

array([b'beforehand gelatine smith dent 12in familiar influence pirate 2 donuts',
       b' released thenplace energ crumblined reap 275 illustration scalds smaller',
       b'constistency belgian incgredients browndo int gaps havent minimarshmellows versa kahluha'],
      dtype=object)

In [32]:
class Translator(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, units,  name_processor, ingredient_processor, target_text_processor):
    super().__init__()
    # Build the encoder and decoder
    encoder = Encoder(name_processor, ingredient_processor, units)
    decoder = Decoder(target_text_processor, units)

    self.encoder = encoder
    self.decoder = decoder

  def call(self, ingredients, names, steps):
    
    context = self.encoder(ingredients, names)
    logits = self.decoder(context, steps)

    #TODO(b/250038731): remove this
    try:
      # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
      del logits._keras_mask
    except AttributeError:
      pass

    return logits

In [33]:
model = Translator(UNITS, name_vectorize_layer, ingredient_vectorize_layer, step_vectorize_layer)

logits = model(ex_ingredients_tok, ex_names_tok, ex_steps_tok)
#logits = model()
print(f'Name tokens, shape: (batch, s, units) {ex_names_tok.shape}')
print(f'Ingredients tokens, shape: (batch, s, units) {ex_ingredients_tok.shape}')
print(f'Steps tokens, shape: (batch, t) {ex_steps_tok.shape}')
print(f'logits, shape: (batch, t, target_vocabulary_size) {logits.shape}')

Dimensions of x1: (64, 4, 256)
Dimensions of x2: (64, 4, 256)
Dimensions of x after embedding: (64, 4, 512)
Dimensions of x after rnn: (64, 4, 256)
Name tokens, shape: (batch, s, units) (64, 4)
Ingredients tokens, shape: (batch, s, units) (64, 4)
Steps tokens, shape: (batch, t) (64, 400)
logits, shape: (batch, t, target_vocabulary_size) (64, 400, 8257)


In [34]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [35]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [36]:
vocab_size = 1.0 * step_vectorize_layer.vocabulary_size()

{"expected_loss": tf.math.log(vocab_size).numpy(),
 "expected_acc": 1/vocab_size}

{'expected_loss': 9.018817, 'expected_acc': 0.00012110936175366356}

In [37]:
model.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [39]:
#model.evaluate(val_ds, steps=20, return_dict=True)

NameError: name 'val_ds' is not defined

In [40]:
history = model.fit(
    train_ds.repeat(), 
    epochs=100,
    steps_per_epoch = 100,
    validation_steps = 20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3)])

NameError: name 'train_ds' is not defined